# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
CV ASSISTANT

In [ ]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests
from IPython.display import Markdown
from pathlib import Path
import time

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

openrouter_url = "https://openrouter.ai/api/v1"

client = OpenAI(api_key=openai_api_key, base_url=openrouter_url)
groq_client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

GPT_MODEL = "openai/gpt-5-nano"
GEMINI_MODEL = "google/gemini-2.5-pro-preview"

system_message = """
You are an assistant who help writing CV. Consider including information about the user's professional 
experience, education, skills, optional certifications and interests. 
Include only the information that is relevant to the position the candidate is applying for.
Try to make your CV not too long and contain the most important information for the employer.
"""

def transcribe_audio(audio_path, language):
    if audio_path is None:
        return ""

    print(f"Transcribing audio: {audio_path} in language: {language}")

    try:
        if not os.path.exists(audio_path):
            print(f"Error: File does not exist: {audio_path}")
            return "⚠️ The audio file doesn't exist"

        file_size = os.path.getsize(audio_path)
        print(f"Audio file size: {file_size} bytes")
        
        if file_size == 0:
            print("Error: Audio file is empty")
            return "⚠️ The audio file is empty. Try record again"

        time.sleep(0.5)

        with open(audio_path, 'rb') as audio_file:
            audio_data = audio_file.read()
            print(f"Read {len(audio_data)} bytes from file")
            
            if len(audio_data) == 0:
                return "⚠️ Failed to read audio data"

            from io import BytesIO
            audio_buffer = BytesIO(audio_data)
            audio_buffer.name = "audio.wav"  
            
            transcription = groq_client.audio.transcriptions.create(
                model="whisper-large-v3-turbo",
                file=audio_buffer,
                language=language if language != "auto" else None,
                response_format="text"
            )
        
        print(f"Transcription result: {transcription}")

        if isinstance(transcription, str):
            return transcription
        else:
            return transcription.text if hasattr(transcription, 'text') else str(transcription)
            
    except Exception as e:
        print(f"Transcription error: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return f"⚠️ Transcription error: {str(e)}"

def convert_chatbot_to_openai_format(chatbot_history):
    messages = []
    for user_msg, bot_msg in chatbot_history:
        if user_msg:
            messages.append({"role": "user", "content": user_msg})
        if bot_msg:
            messages.append({"role": "assistant", "content": bot_msg})
    return messages

def chat(message, chatbot_history, selected_model):
    history_messages = convert_chatbot_to_openai_format(chatbot_history)

    messages = [{"role": "system", "content": system_message}] + history_messages + [
        {"role": "user", "content": message}
    ]

    if selected_model == "GPT-5-nano":
        model = GPT_MODEL
    elif selected_model == "gemini-2.5-pro-preview":
        model = GEMINI_MODEL
    else:
        new_history = chatbot_history + [[message, "⚠️ Unknown model."]]
        yield new_history
        return

    try:
        stream = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )

        response = ""
        for chunk in stream:
            content = chunk.choices[0].delta.content or ""
            response += content

            updated_history = chatbot_history + [[message, response]]
            yield updated_history
            
    except Exception as e:
        print(f"Streaming error: {e}")
        error_history = chatbot_history + [[message, f"⚠️ Error: {str(e)}"]]
        yield error_history

def handle_submit(message, audio, language, history, model):
    if history is None:
        history = []

    if audio is not None:
        print(f"Processing audio: {audio}")
        print(f"Audio type: {type(audio)}")

        audio_path = audio
        if isinstance(audio, tuple):
            print("Audio is tuple format, extracting path...")
            audio_path = audio[1] if len(audio) > 1 else audio[0]
        
        transcribed_text = transcribe_audio(audio_path, language)
        
        if transcribed_text and not transcribed_text.startswith("⚠️"):
            message = transcribed_text
            print(f"Using transcribed text: {message}")
        else:
            error_history = history + [[f"🎤 Audio", transcribed_text]]
            yield "", None, error_history
            return

    if not message or message.strip() == "":
        yield "", None, history
        return

    print(f"Processing message: {message}")

    for updated_history in chat(message, history, model):
        yield "", None, updated_history

with gr.Blocks(theme=gr.themes.Soft()) as UI:
    gr.Markdown("# 📝 CV-writing AI Assistant")
    gr.Markdown("**Write or speak your CV information!**")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                height=500, 
                type="tuples", 
                label="CV Assistant Chat", 
                value=[]
            )

            with gr.Row():
                audio_input = gr.Audio(
                    sources=["microphone"],
                    type="filepath",
                    label="🎤 Click to record, then click STOP, then click Send",
                    show_label=True,
                    interactive=True,
                    format="wav"  
                )

            with gr.Row():
                entry = gr.Textbox(
                    label="Or type your message here",
                    placeholder="I am a software developer with 5 years of experience..",
                    lines=3
                )

            with gr.Row():
                submit_button = gr.Button("✉ Send", variant="primary", scale=2)
                clear_button = gr.Button("🗑️ Clear", scale=1)

        with gr.Column(scale=1):
            model_dropdown = gr.Dropdown(
                choices=["GPT-5-nano", "gemini-2.5-pro-preview"],
                value="GPT-5-nano",
                label="🤖 Choose AI Model"
            )
            language_dropdown = gr.Dropdown(
                choices=[
                    ("🇵🇱 Polski", "pl"),      
                    ("🇬🇧 English", "en"),
                    ("🇩🇪 Deutsch", "de"),
                ],
                value="en",
                label="🌍 Audio Language"
            )

            gr.Markdown("""
                ### How to use:
                
                **Option 1: Voice** 🎤
                1. Click microphone icon to START recording
                2. Speak your CV info clearly
                3. Click STOP to end recording
                4. Click Send button
                
                **Option 2: Text** ⌨️
                1. Type your message
                2. Click Send or press Enter
                
                ### Tips:
                - Speak clearly and not too fast
                - Wait for recording to fully stop before sending
                - If transcription fails, try recording again
            """)

    submit_button.click(
        handle_submit,
        inputs=[entry, audio_input, language_dropdown, chatbot, model_dropdown],
        outputs=[entry, audio_input, chatbot]
    )
    
    entry.submit(
        handle_submit,
        inputs=[entry, audio_input, language_dropdown, chatbot, model_dropdown],
        outputs=[entry, audio_input, chatbot]
    )
    
    clear_button.click(
        lambda: ("", None, []),
        inputs=None,
        outputs=[entry, audio_input, chatbot],
        queue=False
    )

UI.launch()